# Kubernetes Services

# Exposing our application as a Service

## Service abstracts pods details
As mentioned earlier exposing our application as a Service allows to access the application at a single virtual ip address and port - without having to know the ip addresses of the individual pods which may change ip address as pods are scaled up/down or as pods or nodes fail or come on line.

It can also allow load-balancing across pods.

## Accessing our application - via a Service NodePort

We can expose our service using different types.

The first type we will consider is called NodePort and has the effect of exposing our Service on the same port on each Kubernetes node - and so with the ip address of that node.

Let's try that with our service:

In [1]:
. ./.shell.functions

Loading bash helper functions, new functions added:
-- checkPresent 
-- checkState 
-- cont 
-- untilNotPresent 
-- untilNotState 
-- untilPresent 
-- untilState 


In [ ]:
checkPresent svc k8s-demo-service &&
    kubectl delete svc/k8s-demo-service
    
kubectl expose deploy k8s-demo --type=NodePort --name=k8s-demo-service

In [22]:
kubectl get svc/k8s-demo-service -o wide

EXPOSED_PORT=$(kubectl get --no-headers=true svc/k8s-demo-service | awk '{ FS=":"; $0=$5; FS="/"; $0=$2; print $1;}')
CLUSTER_IP=$(kubectl get --no-headers=true svc/k8s-demo-service | awk '{ print $3;}')

echo; echo "----"
echo EXPOSED_PORT=$EXPOSED_PORT
echo CLUSTER_IP=$CLUSTER_IP

NAME               TYPE       CLUSTER-IP       EXTERNAL-IP   PORT(S)          AGE       SELECTOR
k8s-demo-service   NodePort   10.106.186.127   <none>        8080:31420/TCP   15m       app=k8s-demo

----
EXPOSED_PORT=31420
CLUSTER_IP=10.106.186.127


In [24]:
kubectl scale --replicas=3 deploy/k8s-demo

deployment "k8s-demo" scaled


In [25]:
kubectl get all -o wide

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   3         3         3            3           21m       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-6b76bd84d7   3         3         3         21m       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=2632684083

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   3         3         3            3           21m       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-6b76bd84d7   3         3         3         21m       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=2632684083

NAME              

In [101]:
curl $MASTER_IP:$EXPOSED_PORT

curl: (7) Failed to connect to 10.192.0.2 port 31092: Connection refused


: 7

**Note**: We can't connect to the NodePort on the Master because
- No Pods are scheduled to the Master (we would need to taint our Master to allow this)

In [26]:
kubectl describe nodes | grep InternalIP:

MASTER_IP=$(kubectl describe node kube-master | grep InternalIP: | sed 's/.*: *//')
NODE1_IP=$(kubectl describe node kube-node-1 | grep InternalIP: | sed 's/.*: *//')
NODE2_IP=$(kubectl describe node kube-node-2 | grep InternalIP: | sed 's/.*: *//')

echo "MASTER_IP=$MASTER_IP"
echo "NODE1_IP=$NODE1_IP"
echo "NODE2_IP=$NODE2_IP"

  InternalIP:  10.192.0.2
  InternalIP:  10.192.0.3
  InternalIP:  10.192.0.4
MASTER_IP=10.192.0.2
NODE1_IP=10.192.0.3
NODE2_IP=10.192.0.4


In [31]:
curl $NODE2_IP:$EXPOSED_PORT




                               
                    (((((((((                  
               .(((((((((((((((((.             
           .((((((((((((&((((((((((((.         
       /((((((((((((((((@((((((((((((((((/     
      ((((((((((((((((((@((((((((((((((((((    
     *(((((##((((((@@@@@@@@@@@((((((%#(((((*   
     (((((((@@@(@@@@#((@@@((#@@@@(@@@(((((((   
    *(((((((((@@@@(((((@@@(((((@@@@(((((((((,  
    (((((((((@@@%@@@@((@@@((@@@@%@@@(((((((((  
   .(((((((((@@((((@@@@@@@@@@@((((@@(((((((((. 
   (((((((((&@@(((((@@@(((@@@(((((@@&((((((((( 
   (((((((((&@@@@@@@@@@#(#@@@@@@@@@@&((((((((( 
  ((((((@@@@@@@@(((((@@@@@@@(((((&@@@@@@@((((((
  (((((((((((%@@((((%@@@(@@@%((((@@&(((((((((((
   ((((((((((((@@@((@@%(((%@@((@@@(((((((((((( 
     (((((((((((#@@@@%(((((&@@@@#(((((((((((   
      /(((((((((((@@@@@@@@@@@@@(((((((((((/    
        (((((((((@@(((((((((((@@(((((((((      
          (((((((&(((((((((((((&(((((((        
           /(((((((((((((((((((((((((/         
     

In [32]:
curl $NODE1_IP:$EXPOSED_PORT




                               
                    (((((((((                  
               .(((((((((((((((((.             
           .((((((((((((&((((((((((((.         
       /((((((((((((((((@((((((((((((((((/     
      ((((((((((((((((((@((((((((((((((((((    
     *(((((##((((((@@@@@@@@@@@((((((%#(((((*   
     (((((((@@@(@@@@#((@@@((#@@@@(@@@(((((((   
    *(((((((((@@@@(((((@@@(((((@@@@(((((((((,  
    (((((((((@@@%@@@@((@@@((@@@@%@@@(((((((((  
   .(((((((((@@((((@@@@@@@@@@@((((@@(((((((((. 
   (((((((((&@@(((((@@@(((@@@(((((@@&((((((((( 
   (((((((((&@@@@@@@@@@#(#@@@@@@@@@@&((((((((( 
  ((((((@@@@@@@@(((((@@@@@@@(((((&@@@@@@@((((((
  (((((((((((%@@((((%@@@(@@@%((((@@&(((((((((((
   ((((((((((((@@@((@@%(((%@@((@@@(((((((((((( 
     (((((((((((#@@@@%(((((&@@@@#(((((((((((   
      /(((((((((((@@@@@@@@@@@@@(((((((((((/    
        (((((((((@@(((((((((((@@(((((((((      
          (((((((&(((((((((((((&(((((((        
           /(((((((((((((((((((((((((/         
     

In [33]:
kubectl describe svc/k8s-demo-service

Name:                     k8s-demo-service
Namespace:                default
Labels:                   app=k8s-demo
Annotations:              <none>
Selector:                 app=k8s-demo
Type:                     NodePort
IP:                       10.106.186.127
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  31420/TCP
Endpoints:                10.192.2.18:8080,10.192.3.25:8080,10.192.3.26:8080
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


#### For more information about using NodePort see [HERE](https://kubernetes.io/docs/tasks/access-application-cluster/service-access-application-cluster/)

**Note**: Although there is a Cluster IP there is no Service IP assigned, because we are using NodePort

So we can access our Pods by connecting to the hosting node but not to the Service ... so far ...

## Accessing our application - via a Service LoadBalancer


In [55]:
#kubectl delete svc/k8s-demo-service
kubectl delete svc/k8s-demo-service-lb
#kubectl expose deploy k8s-demo --type=LoadBalancer --name=k8s-demo-service-lb

RS_K8SDEMO=$(kubectl get rs | grep k8s-demo | awk '{ print $1;}')

#kubectl expose deploy k8s-demo --type=LoadBalancer --name=k8s-demo-service-lb

kubectl expose rs $RS_K8SDEMO --type="LoadBalancer" --name="k8s-demo-service-lb"

service "k8s-demo-service-lb" deleted
service "k8s-demo-service-lb" exposed


In [56]:
kubectl get svc -o wide

NAME                  TYPE           CLUSTER-IP       EXTERNAL-IP   PORT(S)          AGE       SELECTOR
k8s-demo-service      NodePort       10.106.186.127   <none>        8080:31420/TCP   37m       app=k8s-demo
k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   6s        app=k8s-demo,pod-template-hash=2632684083
kubernetes            ClusterIP      10.96.0.1        <none>        443/TCP          2d        <none>


### LoadBalancer not working with kubeadm/minikube

As described [here](https://forums.manning.com/posts/list/42097.page) currently Kubernetes implementations created with kubeadm (which was used to prepare this cluster) and minikube do not yet implement tLoadBalancer Service.

In [57]:
#untilNotState svc k8s-demo-service-lb pending

k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   8s
k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   17s
k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   26s
k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   35s
k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   43s
k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   52s
k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   1m
k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   1m
k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   1m
k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   1m
k8s-demo-service-lb   LoadBalancer   10.101.53.45     <pending>     8080:31513/TCP   1m
k8s-demo-service-lb   LoadB

In [40]:
kubectl describe svc/k8s-demo-service-lb

LB_EXPOSED_PORT=$(kubectl get --no-headers=true svc/k8s-demo-service-lb | awk '{ FS=":"; $0=$5; FS="/"; $0=$2; print $1;}')
LB_CLUSTER_IP=$(kubectl get --no-headers=true svc/k8s-demo-service-lb | awk '{ print $3;}')

echo
echo LB_EXPOSED_PORT=$LB_EXPOSED_PORT
echo LB_CLUSTER_IP=$LB_CLUSTER_IP
echo MASTER_IP=$MASTER_IP

Name:                     k8s-demo-service-lb
Namespace:                default
Labels:                   app=k8s-demo
Annotations:              <none>
Selector:                 app=k8s-demo
Type:                     LoadBalancer
IP:                       10.101.118.236
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  30225/TCP
Endpoints:                10.192.2.18:8080,10.192.3.25:8080,10.192.3.26:8080
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>

LB_EXPOSED_PORT=30225
LB_CLUSTER_IP=10.101.118.236
MASTER_IP=10.192.0.2


In [41]:
curl ${LB_CLUSTER_IP}:$LB_EXPOSED_PORT

In [111]:
curl ${MASTER_IP}:$EXPOSED_PORT

curl: (7) Failed to connect to 10.192.0.2 port 30298: Connection refused


: 7

In [ ]:
kubectl describe no/kube-master | grep "InternalIP:"

MASTER_IP=$(kubectl describe no/kube-master | grep "InternalIP:" | sed -e 's/.*: *//')
echo MASTER_IP=$MASTER_IP
WORKER1_IP=$(kubectl describe no/kube-node-1 | grep "InternalIP:" | sed -e 's/.*: *//')
echo WORKER1_IP=$WORKER1_IP
WORKER2_IP=$(kubectl describe no/kube-node-2 | grep "InternalIP:" | sed -e 's/.*: *//')
echo WORKER2_IP=$WORKER2_IP

#curl $MASTER_IP:8080
curl $WORKER2_IP:8080
curl $WORKER2_IP:$EXPOSED_PORT
curl $WORKER1_IP:8080
curl $WORKER1_IP:$EXPOSED_PORT
curl $MASTER_IP:$EXPOSED_PORT

#### For more information about using LoadBalancer see [HERE](https://kubernetes.io/docs/tasks/access-application-cluster/load-balance-access-application-cluster/)
